In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# LCLS Classic model 

In [8]:
from LCLS.bmad import LCLSTaoModel
from LCLS.epics import epics_proxy

import numpy as np
import os
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
# Point to a LCLS tao model
assert 'LCLS_LATTICE' in os.environ
modeldir = os.path.join(os.environ['LCLS_LATTICE'], 'bmad/model/')
initfile = modeldir+'tao.init'

In [10]:
# Cached EPICS pv data
epics = epics_proxy('data/epics_snapshot_2018-03-06T12:07:21.000000-08:00.json', verbose=True)

Loaded data/epics_snapshot_2018-03-06T12:07:21.000000-08:00.json with 1191 PVs


In [11]:
M = LCLSTaoModel(input_file=initfile, epics = epics, verbose=False, ploton=False)
print(M)

_______________________________________________
_______________________________________________
LCLS Copper Linac EPICS info

Bunch charge off cathode:  244.8 pC
Laser heater energy      39.03 μJ =>  44.67 keV rms energy spread
Bunch charge in LTU:           177.7 pC
BC1 mean current:              221.1 A
BC2 peak current:             3430.2 A

Linac    Energy (MeV)      Phase (deg)    fudge (%)
L1       238.779        -26.21         99.6
L1X       220.038        -159.7
L2       5000.08        -32.08        102.4
L3       14429.0          -8.0        103.7

FEL Pulse energy :             -0.024 mJ
_______________________________________________


In [7]:
%%tao 
sho var linac_fudge

-------------------------
Tao> sho var linac_fudge
Variable name:  linac_fudge

 Index  Controlled Attributes(s)    Meas         Model        Design  Useit_opt
     1  O_L1_FUDGE[F]           0.0000E+00    9.9863E-01    1.0000E+00       T
     2  O_L2_FUDGE[F]           0.0000E+00    1.0298E+00    1.0000E+00       T
     3  O_L3_FUDGE[F]           0.0000E+00    1.0417E+00    1.0000E+00       T
 Index  Controlled Attributes(s)    Meas         Model        Design  Useit_opt
-------------------------
Tao> 


## Track particles

In [ ]:
%%tao
set beam_init beam_track_end = UEEND
set csr_param n_bin = 40
snparticle 10000
beamon
beamoff

In [ ]:
h5file = '/Users/chrisonian/Code/GitHub/lcls2-live/devel/test1.h5'
M.cmd('write beam -at UEEND '+h5file)

# Plot

In [ ]:
from opmd_beamphysics import bunch_plotting, bunch_tools, bunch_stats
from bokeh.plotting import  show, output_notebook
from bokeh.layouts import column, row
output_notebook(verbose=False, hide_banner=True)
from h5py import File

In [ ]:
#h1.close()
#h2.close()
h1 = File( '/Users/chrisonian/Code/GitHub/lcls2-live/devel/test1.h5', 'r')
h2 = File( '/Users/chrisonian/Code/GitHub/lcls2-live/devel/test2.h5', 'r')
bunch1 = h1['data/1/particles']
bunch2 = h2['data/1/particles']

show(
    row(
        column(    
            bunch_plotting.plot_bunch_h5(bunch1, 't', 'pz', liveOnly=True,  bins = 100),
            bunch_plotting.plot_histogram_h5(bunch1, 't', bins=100)), 
    column(
        
    bunch_plotting.plot_bunch_h5(bunch2, 't', 'pz', liveOnly=True,  bins = 100),
    bunch_plotting.plot_histogram_h5(bunch2, 't', bins=100))        

    )
    
    )

In [ ]:
%%tao
sho beam UEEND